In [1]:
import rubrix as rb
import pandas as pd

df = pd.read_parquet('/media/datastore/nav_open/nav_all_fast.parquet')



/home/radbrt/anaconda3/envs/rubrix/lib/python3.10/site-packages/jose/backends/cryptography_backend.py:18: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes, int_to_bytes


In [2]:
from janitor import clean_names
df = clean_names(df)

In [3]:
df.columns

Index(['stillingsnummer', 'stillingsnummer_nav_no', 'stilling_id',
       'nav_enhet_kode', 'registrert_dato', 'sistepubl_dato',
       'statistikk_aar_mnd', 'offisiell_statistikk_flagg', 'stilling_kilde',
       'arbeidssted_fylkesnummer', 'arbeidssted_fylke',
       'arbeidssted_kommunenummer', 'arbeidssted_kommune',
       'arbeidssted_landkode', 'arbeidssted_land', 'isco_versjon',
       'yrke_grovgruppe', 'yrkeskode', 'yrke', 'yrkesbetegnelse',
       'bedrift_org_nr', 'bedrift_navn', 'aktiv_flagg',
       'bedrift_naring_primar_kode', 'foretak_org_nr', 'foretak_navn',
       'foretak_sektor_gruppe', 'tilleggskriterium', 'antall_stillinger',
       'stillingstittel'],
      dtype='object')

In [5]:
for i, r in df.head(500).iterrows():
    record = rb.TextClassificationRecord(
        inputs={
            "tittel": r.stillingstittel,
            "gruppe": r.yrke_grovgruppe
        },
        metadata={'bedrift_navn': r.bedrift_navn

        }
    )

    rb.log(record, name = "yrkesgruppe_2", verbose=False)


In [9]:
data = [rb.TextClassificationRecord(inputs={"tittel": r.stillingstittel, "gruppe": r.yrke_grovgruppe}) for i, r in df[['stillingstittel', 'yrke_grovgruppe']].iterrows() if r.stillingstittel is not None and r.yrke_grovgruppe is not None]

In [10]:
rb.log(data, name="full_data", verbose=False)

KeyboardInterrupt: 

In [17]:
rd = rb.load("yrkesgruppe_2", query="status:Validated")

In [18]:
type(rd)

pandas.core.frame.DataFrame

In [20]:
rd.head()

,inputs,prediction,prediction_agent,annotation,annotation_agent,multi_label,explanation,id,metadata,status,event_timestamp,metrics
0,{'gruppe': 'Serviceyrker og annet arbeid (2001...,None,None,office,rubrix,False,None,001d9319-7804-4a91-8849-28b297132ba2,{'bedrift_navn': ' AMEDIA DISTRIBUSJON VIKEN A...,Validated,None,{}
1,"{'gruppe': 'Helse, pleie og omsorg (2001-2011)...",None,None,hybrid,rubrix,False,None,006e3d07-1f0d-4350-98f8-d5c5393bd4b3,{},Validated,None,{}
2,"{'gruppe': 'Undervisning (2001-2011)', 'tittel...",None,None,office,rubrix,False,None,00894a86-18f7-4af9-90a2-ec65be61e2cb,{'bedrift_navn': ' KLYPEN BARNEHAGE'},Validated,None,{}
3,"{'gruppe': 'Helse, pleie og omsorg (2001-2011)...",None,None,office,rubrix,False,None,014694ea-aec1-4c58-81bc-e6fea13deabf,{},Validated,None,{}
4,"{'gruppe': 'Ledere (2001-2011)', 'tittel': 'Ve...",None,None,office,rubrix,False,None,01571f2b-66d2-4189-a7a3-950981b83f48,{'bedrift_navn': ' NORSKILT TRAFFIC SAFETY AS ...,Validated,None,{}


In [21]:
rd.columns

Index(['inputs', 'prediction', 'prediction_agent', 'annotation',
       'annotation_agent', 'multi_label', 'explanation', 'id', 'metadata',
       'status', 'event_timestamp', 'metrics'],
      dtype='object')